In [1]:
import os
import dask.dataframe as dd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer, mean_squared_error
import pyarrow.parquet as pq
import pandas as pd
import datetime

print('Ok - import')

Ok - import


In [2]:
# ---------------- Przygotowanie danych ----------------
competition_path = "M:/PycharmProjects/ds-zt-LGBMRegressor-main/kaggle-dtset"  # Ścieżka do danych konkursowych
batch_size = 1000000  # Rozmiar batcha treningowych
percent_of_dataset = 0.2  # Ile procent dataset używamy

# Ścieżki do danych
train_path = os.path.join(competition_path, "train.parquet")
test_path = os.path.join(competition_path, "test.parquet")
features = [f'feature_{i:02}' for i in range(79)]
target_train = 'responder_6'  # Target do przewidzenia
weights = 'weight'

# Wczytanie danych przy użyciu Dask
ddf_train = dd.read_parquet(train_path, columns=features + [target_train, weights])
data_train = ddf_train.sample(frac=percent_of_dataset, random_state=42).compute()
data_train = data_train.dropna(subset=features + [target_train, weights])  # Usunięcie braków

X_train = data_train[features]
y_train = data_train[target_train]
sample_weights_train = data_train[weights]

print(f"Próbka danych: {data_train.shape}")


Próbka danych: (7073668, 81)


In [ ]:
# ---------------- Parametry GridSearchCV ----------------
param_grid = {
    'n_estimators': [100, 300, 500],       # Liczba drzew
    'learning_rate': [0.01, 0.05, 0.1],    # Współczynnik uczenia
    'num_leaves': [31, 50, 100],           # Liczba liści
    'feature_fraction': [0.8, 0.9, 1.0],   # Frakcja cech używanych na drzewo
    'subsample': [0.8, 0.9, 1.0],          # Frakcja próbek
    'max_depth': [-1, 7, 15],              # Maksymalna głębokość drzewa
    'min_child_samples': [20, 30, 50]      # Minimalna liczba próbek w liściu
}

# Inicjalizacja modelu
model = LGBMRegressor(objective='regression', metric='mse', random_state=42)

# Skorer dla GridSearchCV
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Cross-validation z podziałem czasowym
tscv = TimeSeriesSplit(n_splits=5)

# Inicjalizacja GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=tscv,
    verbose=1,
    n_jobs=-1
)

# Dopasowanie modelu
grid_search.fit(X_train, y_train, sample_weight=sample_weights_train)


Fitting 5 folds for each of 2187 candidates, totalling 10935 fits


In [ ]:
# ---------------- Wyniki GridSearchCV ----------------
print("\nNajlepsze parametry:")
print(grid_search.best_params_)

print("\nNajlepszy wynik (MSE):")
print(-grid_search.best_score_)

# Wyświetlenie wyników dla każdej kombinacji parametrów
print("\nWyniki wszystkich prób:")
results = grid_search.cv_results_
for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"MSE: {-mean:.4f} (std: {std:.4f}) dla parametrów: {params}")

# Zapis wyników do pliku
now = datetime.datetime.now()
date_str = now.strftime("%Y-%m-%d-%H-%M-%S")
file_name = f"gridsearch_results_{date_str}.txt"

with open(file_name, 'w') as f:
    f.write(f"Najlepsze parametry: {grid_search.best_params_}\n")
    f.write(f"Najlepszy wynik (MSE): {-grid_search.best_score_:.4f}\n\n")
    f.write("Wyniki wszystkich prób:\n")
    for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
        f.write(f"MSE: {-mean:.4f} (std: {std:.4f}) dla parametrów: {params}\n")

print(f"\nWyniki zapisano do pliku: {file_name}")